# Global static map of flight routes – the QGIS approach
In this tutorial we will create a global map showing the connections between major airports around the world. You will practice:
1. Layouting a world map
2. Projections and their effect on a world map
3. Tricks for visualizing an over-crowded map.
4. Keep working with data-driven styling

### Data
We will work with a dataset from [Openflights.org](https://openflights.org/). It is a rich database of airports around the world as well as information such as the geographical locations and global connections. For this tutorial, the data has been slightly preprocessed from the raw text files to geospatial point layers – flight route *sources* and *destinations*. An id field named *route_id* connects each route. 

- Download the [tutorial data here](data/sources_and_destinations.gpkg).

### QGIS-files
You can download all of this week's QGIS materials as a zipfile [here](QGIS-files/QGIS-files.zip)

The files relevant for this tutorial are:

#### Style files
These files are used for the examples in this tutorial. A note above the map will tell you which styles are needed for that map:
- [airports_style.qml](QGIS-files/airports_style.qml)
- [flight_routes_style.qml](QGIS-files/flight_routes_style.qml)

#### Model files
This processing model file automates the processing steps taken in this tutorial and outputs layers styled the same as in the example maps.
- [local_map_PPGIS.model3](QGIS-files/flight_routes_and_airports.model3)

## Preprocessing steps


Open the two layers *sources* and *destinations* in *sources_and_destinations.gpkg* in QGIS.

The layers have three fields we'll be using:

- *source / destination_airport*: Unique three character identifier.
- *name_source/\_destination*: Name of the airport.
- *route_id*: Unique identifier for each route. E.g. route with id **1**'s source is **ASF** and destination **KZN**.

A naïve point map doesn't tell us much anything except maybe that there are few airports in the sparsely populated areas of the globe.

<div style="background-color: #39f98f; padding: 10px; border-left: 5px solid #059445; margin-bottom: 10px;">

No style file is provided for this map.
    
</div>

![Practical 2 starting point](Figures/practical2_starting_point.jpeg)

We'll create two layers from this starting point:

- Lines connecting each source and destination route
- Airport layer with information of how many routes terminate at that airport.

And of course create a neat map with these layers!

### Route lines


<div style="background-color: #66f2ff; padding: 10px; border-left: 5px solid #00a4b3; margin-bottom: 10px;">
   
1. Run the processing algorithm *Join by lines (hub lines)*.
2. For the parameters:
    - Hub layer & Spoke layer: *Destinations* and *Sources*
    - Hub / Spoke ID field: *route_id*
    - Select *Create geodesic lines.*
    - Under **Advanced parameters**
        - Distance between vertices: smaller than the default 1000, for example *100*.
        - Select *Split lines at antimeridian...*
3. NB! This tool with this data is really slow! Expect the processing to take about 6 minutes.
    
</div>

You may wonder what geodesic lines are. Basically, they are the shortest possible lines between two points on a surface – also known as great circle lines when calculated on the globe (see [here for a shorth introduction](http://www.blog-geographica.com/2015/06/11/geodesic-lines-gis/) by Cayetano Benavent). Don't believe they are the shortest possible lines? Try changing the project's CRS to a Gnomonic projection (e.g., EPSG:102034) and notice that these curved great circle lines are now straight!

If you want, you may also rerun the tool without selecting geodesic lines (expect it to take another several minutes).

We created these lines as geodesic both to make the routes more accurate to those the airlines may fly and more visually pleasing / less clustered. Nonetheless, the routes are hard to interpret with the default styles.

#### Styling the route lines
As always, we encourage experimentation. One example approach is shared in the style file *flight_routers_style.qml*. 

Some considerations that went into this style:

- Reduced line width (0.1 millimeters) to mitigate crowding.
- Added very high levels of transparency (Opacity: 5 %). Emphasizes areas with many criss-crossing routes such as Central Europe.
- Grey color approaching white (`#c4c4c4`): Good and striking contrast against a dark background – always a strong, if safe, choice!

### Airports
Let's get a count of how many routes terminate in each airport and add that information to our map.

For that, we'll group the values based on a field value and calculate new values based on an *aggregate* function.

<div style="background-color: #66f2ff; padding: 10px; border-left: 5px solid #00a4b3; margin-bottom: 10px;">
   
1. Run the processing tool *Aggregate*.
2. Parameters:
    - Input layer: *Destinations*
    - Group by expression: *destination_airport*
    - Aggregates:
        1. Keep only *destination_airport* and *route_id*, remove others.
        2. Aggregate function
            - destination_airport: *first_value*
            - route_id: *count*
        3. Name:
            - route_id: *routes_count*
    
</div>

#### Styling the airports layer

Using the example style *airports_style.qml*

Ideas behind this style:

- A graduated style by size; airports with more terminations are bigger.
    - 10 classes (sizes 0.5–3.5 mm), classified with Natural breaks (Jenks). This is an arbitrary choice: there could be more classes or fewer; discerning between exact class boundaries is next to impossible in any case.
- Color by the longitude coordinate of the airport.
    - Data-driven coloring under Symbol > Configure symbol > Simple marker > Fill color > Data-driven expression / Assistant. Source `x(@geometry)`. Values from -180 to 180 and Viridis color ramp.
        - Again, this decision is arbitrary and mostly serves to make the map more visually interesting. Another approach would be to have the color carry a meaning; e.g., the color is also dependent on the number of terminations or the color is based on the continent of the airport.
    

## Mapping

Put the layers together and layout a map.

<div style="background-color: #ffa64d; padding: 10px; border-left: 5px solid #cc6600; margin-bottom: 10px;">

Some elements of the example map below:

- **Page**: The data is wider than taller. The width is the A4 standard (297 mm) but the height is reduced to 160 mm. Background set to a very dark gray.
- **Legend**: Manually culled to remove unncessary layers and symbols from the airport layer. Placed in a mostly empty corner of the map – use the whitespace to your advantage!
- **Title**: Something short and informative, placed and sized prominently.

Think of ways to improve the map. For example:
 - **Projection**: the example map uses the standard EPSG:4326 CRS. Would another projection, such as Robinson be more suitable?
 - **Additional layers**: You may also consider adding a world map of country borders into the map, although the structure of land areas emerges quite nicely from just the airports and their connections.
 - **Different cropping**: Try cropping the map to a particular area of the world. What do you have to consider when moving from a global to a regional level?
    
</div>

<div style="background-color: #39f98f; padding: 10px; border-left: 5px solid #059445; margin-bottom: 10px;">

**Style files**

Route layer: *flight_routes_style.qml*
    
Airport layer: *airports_style.qml*

    
</div>

![Air routes example map](Figures/practical2_air_routes.jpeg)

## Replicating the processing flow of this notebook
*flight_routes_and_airports.model3* processing model automates the preprocessing steps of this tutorial. To run it successfully, you will need:
- *flight_routes_style.qml*
- *airports_style.qml*
- *sources_and_destinations.gpkg*

You can download this week's QGIS materials as a zipfile [here](QGIS-files/QGIS-files.zip)

Set the appropriate source and destination layers and the style files. The other parameters should be correct by default

![Practical 2 flights model](Figures/practical2_flights_model.png)